#TEST

In [ ]:
import cv2
import numpy as np
from google.colab.patches import cv2_imshow  # Import cv2_imshow from google.colab.patches


In [ ]:
image = cv2.imread('/content/drive/MyDrive/ML1/Skin_color_predictiion/Test/faces/faces_daatset/white/PXL_20220922_162746982.jpg')

In [ ]:
rgb_image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
hsv_image = cv2.cvtColor(image, cv2.COLOR_BGR2HSV)
ycbcr_image = cv2.cvtColor(image, cv2.COLOR_BGR2YCrCb)


In [ ]:
# Example ROI coordinates (x, y, width, height)
roi_x, roi_y, roi_width, roi_height = 100, 100, 200, 200
roi = hsv_image[roi_y:roi_y + roi_height, roi_x:roi_x + roi_width]


In [ ]:
mean_skin_color = np.mean(roi, axis=(0, 1))


In [ ]:
mean_hue = mean_skin_color[0]


In [ ]:
if 5 <= mean_hue <= 20:
    print("The image contains a face with a skin tone within the specified range.")
else:
    print("The image does not contain a face with a skin tone within the specified range.")


The image contains a face with a skin tone within the specified range.


In [ ]:

cv2.rectangle(image, (roi_x, roi_y), (roi_x + roi_width, roi_y + roi_height), (0, 255, 0), 2)
cv2_imshow(image)
cv2.waitKey(0)
cv2.destroyAllWindows()


In [ ]:
import cv2
import numpy as np

# Load the image
image = cv2.imread('/content/drive/MyDrive/ML1/Skin_color_predictiion/Test/faces/faces_daatset/white/PXL_20220922_162746982.jpg')
rgb_image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)

In [ ]:
# Calculate the mean RGB values of the entire image
mean_r = np.mean(rgb_image[:, :, 0])
mean_g = np.mean(rgb_image[:, :, 1])
mean_b = np.mean(rgb_image[:, :, 2])

In [ ]:
white_threshold = 200  # Adjust as needed
fair_threshold = 180  # Adjust as needed
light_brown_threshold = 150  # Adjust as needed
brown_threshold = 120  # Adjust as needed
dark_brown_threshold = 90  # Adjust as needed

In [ ]:
if mean_r >= white_threshold and mean_g >= white_threshold and mean_b >= white_threshold:
    skin_tone = "White"
elif mean_r >= fair_threshold and mean_g >= fair_threshold and mean_b >= fair_threshold:
    skin_tone = "Fair"
elif mean_r >= light_brown_threshold and mean_g >= light_brown_threshold and mean_b >= light_brown_threshold:
    skin_tone = "Light Brown"
elif mean_r >= brown_threshold and mean_g >= brown_threshold and mean_b >= brown_threshold:
    skin_tone = "Brown"
elif mean_r >= dark_brown_threshold and mean_g >= dark_brown_threshold and mean_b >= dark_brown_threshold:
    skin_tone = "Dark Brown"
else:
    skin_tone = "Black"



In [ ]:
print(f"Skin Tone: {skin_tone}")

Skin Tone: Light Brown


#PRe_Trained_Model

In [ ]:
import cv2
img = cv2.imread(image_path)
img = cv2.resize(img, (224, 224))
img = img / 255.0  # Normalize to [0, 1]



In [ ]:
from tensorflow.keras.preprocessing.image import ImageDataGenerator

datagen = ImageDataGenerator(
    rotation_range=20,
    width_shift_range=0.2,
    height_shift_range=0.2,
    horizontal_flip=True,
    vertical_flip=True,
    zoom_range=0.2,
    shear_range=0.2,
    fill_mode='nearest'
)


In [ ]:
from tensorflow.keras.applications import MobileNetV2
from tensorflow.keras.layers import GlobalAveragePooling2D, Dense
from tensorflow.keras.models import Model

# Load pre-trained MobileNetV2 model
base_model = MobileNetV2(weights='imagenet', include_top=False)


In [ ]:
# Add a Global Average Pooling layer
x = base_model.output
x = GlobalAveragePooling2D()(x)

# Add one or more fully connected (dense) layers
x = Dense(128, activation='relu')(x)  # You can adjust the number of units
predictions = Dense(9, activation='softmax')(x)  # 9 classes for skin tones


In [ ]:
model = Model(inputs=base_model.input, outputs=predictions)


In [ ]:
# # Freeze layers in the base model
# for layer in base_model.layers:
#     layer.trainable = False


In [ ]:
from tensorflow.keras.optimizers import Adam

model.compile(optimizer=Adam(learning_rate=0.001),
              loss='categorical_crossentropy',
              metrics=['accuracy'])


In [ ]:
history = model.fit(X_train, y_train, epochs=10, validation_data=(X_val, y_val))


In [ ]:
test_loss, test_accuracy = model.evaluate(X_test, y_test)


In [ ]:
# Make predictions on new images
predictions = model.predict(new_images)
